In [2]:
# ------------------------- Incidents -----------------------------
import pandas as pd
import numpy as np

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 2 dataset
df1 = pd.read_csv(
    "../data/LFB Incident data from 2009 - 2017.csv", sep=",", low_memory=False
)
df2 = pd.read_csv(
    "../data/LFB Incident data from 2018 onwards.csv", sep=",", low_memory=False
)

In [ ]:
# concaténation
df = pd.concat([df1, df2], axis=0)
# reset index
df = df.reset_index(drop=True)

# suppression des ".00" sur certaines lignes de "IncidentNumber"
df["IncidentNumber"] = df["IncidentNumber"].apply(
    lambda x: str(x)[0:-3] if (".00" in x) & (str(x)[-3:] == ".00") else x
)

# formatage des dates
df.DateOfCall = pd.to_datetime(df.DateOfCall, format="%d-%b-%y")
df.TimeOfCall = pd.to_datetime(df.TimeOfCall, format="%H:%M:%S")

# création de champs Mois et DayOfWeek
df["Month"] = df["DateOfCall"].dt.month
df["DayOfWeek"] = df["DateOfCall"].dt.dayofweek + 1

display(df.head())
display(df.info())

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls,Month,DayOfWeek
0,235138081,2009-01-01,2009,1900-01-01 00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,NaN,NaN,E09000032,WANDSWORTH,Wandsworth,E05014010,Battersea Park,Battersea Park,528652.0,176830.0,528650,176850,51.475812,-0.148894,London,Battersea,319.0,Battersea,342.0,Clapham,2.0,2.0,2,60,255,1.0,1,4
1,1091,2009-01-01,2009,1900-01-01 00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,Open land/water - nearest gazetteer location,SE1 7SG,SE1,NaN,NaN,E09000022,LAMBETH,Lambeth,E05014118,Waterloo & South Bank,Waterloo & South Bank,530485.0,179007.0,530450,179050,51.494957,-0.121712,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0,1,4
2,2091,2009-01-01,2009,1900-01-01 00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,In street outside gazetteer location,N9 9EL,N9,NaN,NaN,E09000010,ENFIELD,Enfield,E05013682,Haselbury,Haselbury,533773.0,194492.0,533750,194450,51.633342,-0.068488,London,Edmonton,308.0,Edmonton,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
3,3091,2009-01-01,2009,1900-01-01 00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),On land associated with building,UB10 0DG,UB10,1.000215e+11,21401491.0,E09000017,HILLINGDON,Hillingdon,E05013571,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,51.533882,-0.448089,London,Hillingdon,210.0,Hillingdon,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
4,5091,2009-01-01,2009,1900-01-01 00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,In street outside gazetteer location,N7 8HG,N7,NaN,NaN,E09000019,ISLINGTON,Islington,E05013708,Laycock,Laycock,531058.0,185307.0,531050,185350,51.551441,-0.111120,London,Holloway,233.0,Holloway,250.0,Holloway,1.0,2.0,2,60,255,1.0,1,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 41 columns):
 #   Column                                  Dtype         
---  ------                                  -----         
 0   IncidentNumber                          object        
 1   DateOfCall                              datetime64[ns]
 2   CalYear                                 int64         
 3   TimeOfCall                              datetime64[ns]
 4   HourOfCall                              int64         
 5   IncidentGroup                           object        
 6   StopCodeDescription                     object        
 7   SpecialServiceType                      object        
 8   PropertyCategory                        object        
 9   PropertyType                            object        
 10  AddressQualifier                        object        
 11  Postcode_full                           object        
 12  Postcode_district                       ob

None

In [ ]:
def TestDoublonSousCatégorie(categorie, ssCategorie):
    # Vérifie si les StopCodeDescription existe en double dans les IncidentGroup
    result = df.groupby(ssCategorie)[categorie].nunique()
    print(result[result.values > 1])
    print("Na pour", categorie, df[categorie].isna().sum())
    print("Na pour", ssCategorie, df[ssCategorie].isna().sum())


TestDoublonSousCatégorie("IncidentGroup", "StopCodeDescription")
print(
    "\n",
    "6 na sur IncidentGroup et pas de doublon, peut supprimer la colonne IncidentGroup",
)

print()
TestDoublonSousCatégorie("PropertyCategory", "PropertyType")
print(
    "\n",
    "6 na sur PropertyCategory et 6 pour PropertyType, peut supprimer la colonne PropertyCategory",
)


def TreatText(s):
    if not pd.isna(s):
        s = s.upper().strip()
    return s


df.StopCodeDescription = df.StopCodeDescription.apply(lambda s: TreatText(s))
df.SpecialServiceType = df.SpecialServiceType.apply(lambda s: TreatText(s))
df.PropertyType = df.PropertyType.apply(lambda s: TreatText(s))
df.AddressQualifier = df.AddressQualifier.apply(lambda s: TreatText(s))
df.Postcode_district = df.Postcode_district.apply(lambda s: TreatText(s))
df.IncGeo_BoroughCode = df.IncGeo_BoroughCode.apply(lambda s: TreatText(s))
df.IncGeo_WardCode = df.IncGeo_WardCode.apply(lambda s: TreatText(s))
df.FirstPumpArriving_DeployedFromStation = (
    df.FirstPumpArriving_DeployedFromStation.apply(lambda s: TreatText(s))
)

display(df.head())

Series([], Name: IncidentGroup, dtype: int64)
Na pour IncidentGroup 6
Na pour StopCodeDescription 0

 6 na sur IncidentGroup et pas de doublon, peut supprimer la colonne IncidentGroup

Series([], Name: PropertyCategory, dtype: int64)
Na pour PropertyCategory 6
Na pour PropertyType 6

 6 na sur PropertyCategory et 6 pour PropertyType, peut supprimer la colonne PropertyCategory


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls,Month,DayOfWeek
0,235138081,2009-01-01,2009,1900-01-01 00:00:37,0,Special Service,SPECIAL SERVICE,RTC,Road Vehicle,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11 4LB,SW11,NaN,NaN,E09000032,WANDSWORTH,Wandsworth,E05014010,Battersea Park,Battersea Park,528652.0,176830.0,528650,176850,51.475812,-0.148894,London,Battersea,319.0,BATTERSEA,342.0,Clapham,2.0,2.0,2,60,255,1.0,1,4
1,1091,2009-01-01,2009,1900-01-01 00:00:46,0,Special Service,SPECIAL SERVICE,ASSIST OTHER AGENCIES,Outdoor,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1 7SG,SE1,NaN,NaN,E09000022,LAMBETH,Lambeth,E05014118,Waterloo & South Bank,Waterloo & South Bank,530485.0,179007.0,530450,179050,51.494957,-0.121712,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0,1,4
2,2091,2009-01-01,2009,1900-01-01 00:03:00,0,Fire,SECONDARY FIRE,NaN,Outdoor,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9 9EL,N9,NaN,NaN,E09000010,ENFIELD,Enfield,E05013682,Haselbury,Haselbury,533773.0,194492.0,533750,194450,51.633342,-0.068488,London,Edmonton,308.0,EDMONTON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
3,3091,2009-01-01,2009,1900-01-01 00:04:27,0,Fire,SECONDARY FIRE,NaN,Outdoor,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10 0DG,UB10,1.000215e+11,21401491.0,E09000017,HILLINGDON,Hillingdon,E05013571,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,51.533882,-0.448089,London,Hillingdon,210.0,HILLINGDON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
4,5091,2009-01-01,2009,1900-01-01 00:05:39,0,Fire,SECONDARY FIRE,NaN,Outdoor,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7 8HG,N7,NaN,NaN,E09000019,ISLINGTON,Islington,E05013708,Laycock,Laycock,531058.0,185307.0,531050,185350,51.551441,-0.111120,London,Holloway,233.0,HOLLOWAY,250.0,Holloway,1.0,2.0,2,60,255,1.0,1,4


In [ ]:
# Merge de SpecialService dans Stopcode
s = "Special Service".upper().strip()
df["StopCode"] = df.apply(
    lambda row: (
        "SST-" + row["SpecialServiceType"]
        if row["StopCodeDescription"] == s
        else row["StopCodeDescription"]
    ),
    axis=1,
)
display(df.head())
# df.StopCodeDescription
# SpecialServiceType

df = df.drop(["StopCodeDescription", "SpecialServiceType"], axis=1)

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls,Month,DayOfWeek,StopCode
0,235138081,2009-01-01,2009,1900-01-01 00:00:37,0,Special Service,SPECIAL SERVICE,RTC,Road Vehicle,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11 4LB,SW11,NaN,NaN,E09000032,WANDSWORTH,Wandsworth,E05014010,Battersea Park,Battersea Park,528652.0,176830.0,528650,176850,51.475812,-0.148894,London,Battersea,319.0,BATTERSEA,342.0,Clapham,2.0,2.0,2,60,255,1.0,1,4,SST-RTC
1,1091,2009-01-01,2009,1900-01-01 00:00:46,0,Special Service,SPECIAL SERVICE,ASSIST OTHER AGENCIES,Outdoor,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1 7SG,SE1,NaN,NaN,E09000022,LAMBETH,Lambeth,E05014118,Waterloo & South Bank,Waterloo & South Bank,530485.0,179007.0,530450,179050,51.494957,-0.121712,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0,1,4,SST-ASSIST OTHER AGENCIES
2,2091,2009-01-01,2009,1900-01-01 00:03:00,0,Fire,SECONDARY FIRE,NaN,Outdoor,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9 9EL,N9,NaN,NaN,E09000010,ENFIELD,Enfield,E05013682,Haselbury,Haselbury,533773.0,194492.0,533750,194450,51.633342,-0.068488,London,Edmonton,308.0,EDMONTON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4,SECONDARY FIRE
3,3091,2009-01-01,2009,1900-01-01 00:04:27,0,Fire,SECONDARY FIRE,NaN,Outdoor,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10 0DG,UB10,1.000215e+11,21401491.0,E09000017,HILLINGDON,Hillingdon,E05013571,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,51.533882,-0.448089,London,Hillingdon,210.0,HILLINGDON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4,SECONDARY FIRE
4,5091,2009-01-01,2009,1900-01-01 00:05:39,0,Fire,SECONDARY FIRE,NaN,Outdoor,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7 8HG,N7,NaN,NaN,E09000019,ISLINGTON,Islington,E05013708,Laycock,Laycock,531058.0,185307.0,531050,185350,51.551441,-0.111120,London,Holloway,233.0,HOLLOWAY,250.0,Holloway,1.0,2.0,2,60,255,1.0,1,4,SECONDARY FIRE


In [6]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
df = df.drop(
    [
        "DateOfCall",
        "TimeOfCall",
        "IncidentGroup",
        "PropertyCategory",
        "AddressQualifier",
        # Geo
        "Postcode_full",
        "UPRN",
        "USRN",
        "IncGeo_BoroughCode",
        "IncGeo_BoroughName",
        "ProperCase",
        "IncGeo_WardCode",
        "IncGeo_WardName",
        "IncGeo_WardNameNew",
        "Easting_m",
        "Northing_m",
        "Easting_rounded",
        "Northing_rounded",
        "Latitude",
        "Longitude",
        #
        "FRS",
        "IncidentStationGround",
        "FirstPumpArriving_AttendanceTime",
        "SecondPumpArriving_AttendanceTime",
        "SecondPumpArriving_DeployedFromStation",
        "NumStationsWithPumpsAttending",
        "PumpCount",
        "PumpMinutesRounded",
        "Notional Cost (£)",
        "NumCalls",
    ],
    axis=1,
)

In [7]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreIncidents.csv", sep=";", index=False)

In [8]:
# ------------------------- Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 3 dataset
df1 = pd.read_csv(
    "../data/LFB Mobilisation data from January 2009 - 2014.csv",
    sep=";",
    low_memory=False,
)
df2 = pd.read_csv(
    "../data/LFB Mobilisation data from 2015 - 2020.csv", sep=";", low_memory=False
)
df3 = pd.read_csv(
    "../data/LFB Mobilisation data from 2021 - 2024.csv",
    sep=",",
    low_memory=False,
    usecols=lambda column: column not in ["BoroughName", "WardName"],
)

In [ ]:
# concaténation
df = pd.concat([df1, df2, df3], axis=0)
# reset index
df = df.reset_index(drop=True)

# formatage des dates
df.DateAndTimeMobilised = pd.to_datetime(
    df.DateAndTimeMobilised, format="%d/%m/%Y %H:%M"
)
df.DateAndTimeMobile = pd.to_datetime(df.DateAndTimeMobile, format="%d/%m/%Y %H:%M")
df.DateAndTimeArrived = pd.to_datetime(df.DateAndTimeArrived, format="%d/%m/%Y %H:%M")
df.DateAndTimeLeft = pd.to_datetime(df.DateAndTimeLeft, format="%d/%m/%Y %H:%M")
df.DateAndTimeReturned = pd.to_datetime(df.DateAndTimeReturned, format="%d/%m/%Y %H:%M")

display(df.head())
display(df.info())

,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description
0,235138081,2009,0,38426,H271,1,2009-01-01 00:02:00,NaT,2009-01-01 00:07:00,NaN,NaN,319,2009-01-01 00:16:00,2009-01-01 00:23:00,H27,Battersea,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
1,235138081,2009,0,38427,H212,2,2009-01-01 00:02:00,2009-01-01 00:06:00,2009-01-01 00:08:00,253.0,89.0,342,2009-01-01 00:12:00,2009-01-01 00:12:00,H21,Clapham,Home Station,2,Initial,Initial Mobilisation,NaN,NaN
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
3,3091,2009,0,38430,G232,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:08:00,108.0,102.0,210,2009-01-01 00:20:00,2009-01-01 00:20:00,G23,Hillingdon,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
4,5091,2009,0,38432,A311,1,2009-01-01 00:06:00,2009-01-01 00:07:00,2009-01-01 00:09:00,114.0,119.0,233,2009-01-01 00:11:00,2009-01-01 00:22:00,A31,Holloway,Home Station,1,Initial,Initial Mobilisation,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480004 entries, 0 to 2480003
Data columns (total 22 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   IncidentNumber            object        
 1   CalYear                   int64         
 2   HourOfCall                int64         
 3   ResourceMobilisationId    int64         
 4   Resource_Code             object        
 5   PerformanceReporting      object        
 6   DateAndTimeMobilised      datetime64[ns]
 7   DateAndTimeMobile         datetime64[ns]
 8   DateAndTimeArrived        datetime64[ns]
 9   TurnoutTimeSeconds        float64       
 10  TravelTimeSeconds         float64       
 11  AttendanceTimeSeconds     int64         
 12  DateAndTimeLeft           datetime64[ns]
 13  DateAndTimeReturned       datetime64[ns]
 14  DeployedFromStation_Code  object        
 15  DeployedFromStation_Name  object        
 16  DeployedFromLocation      object        
 17  PumpOrde

None

In [ ]:
# création de la colonne Temps sur site en seconde de la pompe par delta de temps arrivée et départ
df["PumpOnSiteSeconds"] = (df.DateAndTimeLeft - df.DateAndTimeArrived).dt.seconds
display(df.head(3))

# Regroupe les lignes par incidents, car il y a une ligne par incidents / camion avec un n° d'ordre
# Comme on s'intéresse à prédire des temps, on calcule les temps par incidents, min, max et moyen
# et on cherchera à prédire des tgemps
# les infos de 1ere station sur place et nb de pompe seront prises à partir de incidents
aggregated = (
    df.groupby("IncidentNumber")
    .agg(
        PumpSecondsOnSite_min=("PumpOnSiteSeconds", "min"),
        PumpSecondsOnSite_mean=("PumpOnSiteSeconds", "mean"),
        PumpSecondsOnSite_max=("PumpOnSiteSeconds", "max"),
        TurnoutTimeSeconds_min=("TurnoutTimeSeconds", "min"),
        TurnoutTimeSeconds_mean=("TurnoutTimeSeconds", "mean"),
        TurnoutTimeSeconds_max=("TurnoutTimeSeconds", "max"),
        TravelTimeSeconds_min=("TravelTimeSeconds", "min"),
        TravelTimeSeconds_mean=("TravelTimeSeconds", "mean"),
        TravelTimeSeconds_max=("TravelTimeSeconds", "max"),
    )
    .reset_index()
)
display(aggregated.head())

# jointure de l'agrégat sur le dataframe pour ajouter les colonnes de temps
merge = pd.merge(
    df, aggregated, left_on="IncidentNumber", right_on="IncidentNumber", how="left"
).sort_values(by=["IncidentNumber", "PumpOrder"])

display(merge.head(10))
print(merge.shape)

,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description,PumpOnSiteSeconds
0,235138081,2009,0,38426,H271,1,2009-01-01 00:02:00,NaT,2009-01-01 00:07:00,NaN,NaN,319,2009-01-01 00:16:00,2009-01-01 00:23:00,H27,Battersea,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,540.0
1,235138081,2009,0,38427,H212,2,2009-01-01 00:02:00,2009-01-01 00:06:00,2009-01-01 00:08:00,253.0,89.0,342,2009-01-01 00:12:00,2009-01-01 00:12:00,H21,Clapham,Home Station,2,Initial,Initial Mobilisation,NaN,NaN,240.0
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,420.0


,IncidentNumber,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1092,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
1,1111,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
2,1112,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
3,1121,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
4,1131,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0


,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description,PumpOnSiteSeconds,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
25229,1092,2009,10,72977,G221,1,2009-02-23 10:02:00,2009-02-23 10:04:00,2009-02-23 10:11:00,128.0,390.0,518,NaT,2009-02-23 10:22:00,G22,Stanmore,Home Station,1,Initial,Initial Mobilisation,12.0,Not held up,NaN,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
338999,1111,2011,0,3610263,A411,1,2011-01-01 00:03:00,2011-01-01 00:04:00,2011-01-01 00:12:00,71.0,480.0,551,2011-01-01 00:14:00,2011-01-01 00:20:00,A41,West Hampstead,Home Station,1,Initial,Initial Mobilisation,12.0,Not held up,120.0,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
346998,1112,2011,1,3621855,E331,1,2011-01-22 01:32:00,2011-01-22 01:34:00,2011-01-22 01:38:00,119.0,225.0,344,2011-01-22 01:46:00,2011-01-22 01:47:00,E33,Southwark,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,480.0,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
492154,1121,2012,0,3831037,G401,1,2012-01-01 00:01:00,2012-01-01 00:02:00,2012-01-01 00:05:00,65.0,192.0,257,2012-01-01 00:11:00,2012-01-01 00:18:00,G40,Hayes,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,360.0,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
492155,1121,2012,0,3831038,G402,2,2012-01-01 00:01:00,2012-01-01 00:02:00,2012-01-01 00:06:00,67.0,235.0,302,2012-01-01 00:10:00,2012-01-01 00:17:00,G40,Hayes,Home Station,2,Initial,Initial Mobilisation,NaN,NaN,240.0,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
637588,1131,2013,0,4036113,A412,1,2013-01-01 00:02:00,2013-01-01 00:03:00,2013-01-01 00:05:00,89.0,78.0,167,2013-01-01 00:11:00,2013-01-01 00:12:00,A41,West Hampstead,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,360.0,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0
733568,1133,2013,12,4179294,G281,1,2013-09-10 11:06:00,2013-09-10 11:06:00,2013-09-10 11:11:00,1.0,267.0,268,2013-09-10 11:14:00,2013-09-10 11:14:00,G28,Willesden,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,180.0,180.0,180.0,180.0,1.0,1.0,1.0,267.0,267.0,267.0
880681,1143,2014,11,4407663,H431,1,2014-10-28 11:57:00,2014-10-28 11:58:00,2014-10-28 12:04:00,42.0,377.0,419,2014-10-28 12:08:00,NaT,H43,Twickenham,Home Station,1,Initial,Initial Mobilisation,9.0,"Traffic, roadworks, etc",240.0,240.0,240.0,240.0,42.0,42.0,42.0,377.0,377.0,377.0
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,420.0,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
25230,2092,2009,10,72978,E241,1,2009-02-23 10:03:00,2009-02-23 10:04:00,2009-02-23 10:06:00,85.0,85.0,170,NaT,2009-02-23 10:17:00,E24,Woolwich,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,NaN,300.0,300.0,300.0,78.0,81.5,85.0,85.0,208.0,331.0


(2480004, 32)


In [11]:
# Pas de traitement particulier sur PlusCode_Code, car les lignes Add et RCA ne sont pas des lignes en doublons d'un Initial, et peuvent donc être traitées comme initial

# suppression des duplicatas pour ne garder qu'une ligne par incidents
merge = merge.drop_duplicates(subset=["IncidentNumber"], keep="first")
merge.IncidentNumber.duplicated().sum()

np.int64(0)

In [12]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
merge = merge.drop(
    [
        "CalYear",
        "HourOfCall",
        "ResourceMobilisationId",
        "Resource_Code",
        "PerformanceReporting",
        "DateAndTimeMobilised",
        "DateAndTimeMobile",
        "DateAndTimeArrived",
        "TurnoutTimeSeconds",
        "TravelTimeSeconds",
        "PumpOnSiteSeconds",
        "AttendanceTimeSeconds",
        "DateAndTimeLeft",
        "DateAndTimeReturned",
        "DeployedFromStation_Code",
        "DeployedFromStation_Name",
        "DeployedFromLocation",
        "PumpOrder",
        "PlusCode_Code",
        "PlusCode_Description",
        "DelayCodeId",
        "DelayCode_Description",
    ],
    axis=1,
)

In [13]:
# sauvegarde d'un fichier temporaire
merge.to_csv("../data/PreMobilisations.csv", sep=";", index=False)

In [ ]:
# ------------------------- Merge Incidents et Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Charge les dataset temporaire
df_incidents = pd.read_csv("../data/PreIncidents.csv", sep=";", low_memory=False)
df_mobilisations = pd.read_csv(
    "../data/PreMobilisations.csv", sep=";", low_memory=False
)

display(df_incidents.head())
display(df_incidents.info())

display(df_mobilisations.head())
display(df_mobilisations.info())

,IncidentNumber,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode
0,235138081,2009,0,CAR,SW11,BATTERSEA,2.0,1,4,SST-RTC
1,1091,2009,0,LAKE/POND/RESERVOIR,SE1,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES
2,2091,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,N7,HOLLOWAY,2.0,1,4,SECONDARY FIRE


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 10 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   Postcode_district                      object 
 5   FirstPumpArriving_DeployedFromStation  object 
 6   NumPumpsAttending                      float64
 7   Month                                  int64  
 8   DayOfWeek                              int64  
 9   StopCode                               object 
dtypes: float64(1), int64(4), object(5)
memory usage: 135.1+ MB


None

,IncidentNumber,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1092,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
1,1111,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
2,1112,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
3,1121,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
4,1131,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642042 entries, 0 to 1642041
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   IncidentNumber           1642042 non-null  object 
 1   PumpSecondsOnSite_min    1612710 non-null  float64
 2   PumpSecondsOnSite_mean   1612710 non-null  float64
 3   PumpSecondsOnSite_max    1612710 non-null  float64
 4   TurnoutTimeSeconds_min   1624689 non-null  float64
 5   TurnoutTimeSeconds_mean  1624689 non-null  float64
 6   TurnoutTimeSeconds_max   1624689 non-null  float64
 7   TravelTimeSeconds_min    1624692 non-null  float64
 8   TravelTimeSeconds_mean   1624692 non-null  float64
 9   TravelTimeSeconds_max    1624692 non-null  float64
dtypes: float64(9), object(1)
memory usage: 125.3+ MB


None

In [ ]:
# Merge
df = pd.merge(
    df_incidents,
    df_mobilisations,
    left_on="IncidentNumber",
    right_on="IncidentNumber",
    how="left",
)
df = df.reset_index(drop=True)

# on vérifie qu'il n'y a pas de doublon sur IncidentNumber avant de la supprimer
print("IncidentNumber na :", df.IncidentNumber.isna().sum())

display(df.head())
display(df.info())

IncidentNumber na : 0


,IncidentNumber,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1091,2009,0,LAKE/POND/RESERVOIR,SE1,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,N7,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 19 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   Postcode_district                      object 
 5   FirstPumpArriving_DeployedFromStation  object 
 6   NumPumpsAttending                      float64
 7   Month                                  int64  
 8   DayOfWeek                              int64  
 9   StopCode                               object 
 10  PumpSecondsOnSite_min                  float64
 11  PumpSecondsOnSite_mean                 float64
 12  PumpSecondsOnSite_max                  float64
 13  TurnoutTimeSeconds_min                 float64
 14  TurnoutTimeSeconds_mean                float64
 15

None

In [16]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreProcessTemp.csv", sep=";", index=False)

In [ ]:
# ------------------------- Gestion des NA -----------------------------

# Nettoyage du fichier PreProcessTemp. Pas de doublon sur IncidentNumber, vérifié juste avant
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/PreProcessTemp.csv", sep=";", low_memory=False)

display(df.head())
display(df.info())


,IncidentNumber,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1091,2009,0,LAKE/POND/RESERVOIR,SE1,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,N7,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 19 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   Postcode_district                      object 
 5   FirstPumpArriving_DeployedFromStation  object 
 6   NumPumpsAttending                      float64
 7   Month                                  int64  
 8   DayOfWeek                              int64  
 9   StopCode                               object 
 10  PumpSecondsOnSite_min                  float64
 11  PumpSecondsOnSite_mean                 float64
 12  PumpSecondsOnSite_max                  float64
 13  TurnoutTimeSeconds_min                 float64
 14  TurnoutTimeSeconds_mean                float64
 15

None

In [ ]:
# Gestion des NA


def ShowNa():
    # Affichage des NA
    na_counts = df.isna().sum()
    print(na_counts[na_counts > 0], "\n")

    print("\ntotal ligne na", df.isna().any(axis=1).sum())
    print("Total de lignes", len(df))
    print("Poucentage", df.isna().any(axis=1).sum() / len(df) * 100, "%")

    print("\nligne par année")
    # display(df.CalYear.value_counts())

    # print("ligne na par année")
    # display(df_temp.loc[df_temp.isna().any(axis=1), "CalYear"].value_counts())
    # print("Les lignes avec des na sont principalments des lignes d'avant 2014")

    # years = []
    # for year in range(df_temp.CalYear.min(), df_temp.CalYear.max() + 1):
    #     total = len(df_temp.loc[df_temp.CalYear == year])
    #     na = df_temp[df_temp.CalYear == year].isna().any(axis=1).sum()
    #     years.append([year, total, na, na / total * 100])

    years = []
    for year in range(df.CalYear.min(), df.CalYear.max() + 1):
        for col in df.columns:
            total = len(df.loc[df.CalYear == year])
            na = df.loc[df.CalYear == year, col].isna().sum()
            if na > 0:
                years.append([year, col, total, na, na / total * 100])

    df_years = pd.DataFrame(
        years, columns=["Année", "Colonne", "Nb lignes", "Na", "%Na"]
    )
    # display(df_years)
    display(df_years.groupby(["Année", "Colonne"]).min())


ShowNa()

# supprime les lignes avec très peu de NA
# les lignes avec des USRS vides datent d'avant 2015, on les supprime
df = df.dropna(axis=0, how="any", subset=["PropertyType", "NumPumpsAttending"])
ShowNa()

# Il n'existe pas toujours de ligne d'intervention pour les lignes d'Incidents, on supprime les lignes sans informations de mobilisation, et parfois l'information est partiellement saisie
df = df.dropna(
    axis=0,
    how="any",
    subset=[
        "PumpSecondsOnSite_min",
        "TurnoutTimeSeconds_min",
        "TravelTimeSeconds_min",
        "FirstPumpArriving_DeployedFromStation",
    ],
)
ShowNa()

PropertyType                                  6
FirstPumpArriving_DeployedFromStation    135700
NumPumpsAttending                         14103
PumpSecondsOnSite_min                    165344
PumpSecondsOnSite_mean                   165344
PumpSecondsOnSite_max                    165344
TurnoutTimeSeconds_min                   153350
TurnoutTimeSeconds_mean                  153350
TurnoutTimeSeconds_max                   153350
TravelTimeSeconds_min                    153347
TravelTimeSeconds_mean                   153347
TravelTimeSeconds_max                    153347
dtype: int64 


total ligne na 179894
Total de lignes 1771413
Poucentage 10.155395720817223 %

ligne par année


Nb lignes     Na        %Na
Année Colonne                                                           
2009  FirstPumpArriving_DeployedFromStation     134379  13173   9.802871
      NumPumpsAttending                         134379   1341   0.997924
      PumpSecondsOnSite_max                     134379  33117  24.644476
      PumpSecondsOnSite_mean                    134379  33117  24.644476
      PumpSecondsOnSite_min                     134379  33117  24.644476
      TravelTimeSeconds_max                     134379  17350  12.911244
      TravelTimeSeconds_mean                    134379  17350  12.911244
      TravelTimeSeconds_min                     134379  17350  12.911244
      TurnoutTimeSeconds_max                    134379  17349  12.910499
      TurnoutTimeSeconds_mean                   134379  17349  12.910499
      TurnoutTimeSeconds_min                    134379  17349  12.910499
2010  FirstPumpArriving_DeployedFromStation     125060  14704  11.757556
      NumPumpsAttending                         125060    672   0.537342
      PumpSecondsOnSite_max                     125060  18337  14.662562
      PumpSecondsOnSite_mean                    125060  18337  14.662562
      PumpSecondsOnSite_min                     125060  18337  14.662562
      TravelTimeSeconds_max                     125060  16541  13.226451
      TravelTimeSeconds_mean                    125060  16541  13.226451
      TravelTimeSeconds_min                     125060  16541  13.226451
      TurnoutTimeSeconds_max                    125060  16556  13.238446
      TurnoutTimeSeconds_mean                   125060  16556  13.238446
      TurnoutTimeSeconds_min                    125060  16556  13.238446
2011  FirstPumpArriving_DeployedFromStation     115132  12448  10.811938
      NumPumpsAttending                         115132    447   0.388250
      PumpSecondsOnSite_max                     115132  15613  13.560956
      PumpSecondsOnSite_mean                    115132  15613  13.560956
      PumpSecondsOnSite_min                     115132  15613  13.560956
      TravelTimeSeconds_max                     115132  14014  12.172115
      TravelTimeSeconds_mean                    115132  14014  12.172115
      TravelTimeSeconds_min                     115132  14014  12.172115
      TurnoutTimeSeconds_max                    115132  14012  12.170378
      TurnoutTimeSeconds_mean                   115132  14012  12.170378
      TurnoutTimeSeconds_min                    115132  14012  12.170378
2012  FirstPumpArriving_DeployedFromStation     108391  11612  10.713067
      NumPumpsAttending                         108391    746   0.688249
      PumpSecondsOnSite_max                     108391  12348  11.392090
      PumpSecondsOnSite_mean                    108391  12348  11.392090
      PumpSecondsOnSite_min                     108391  12348  11.392090
      TravelTimeSeconds_max                     108391  12977  11.972396
      TravelTimeSeconds_mean                    108391  12977  11.972396
      TravelTimeSeconds_min                     108391  12977  11.972396
      TurnoutTimeSeconds_max                    108391  12971  11.966861
      TurnoutTimeSeconds_mean                   108391  12971  11.966861
      TurnoutTimeSeconds_min                    108391  12971  11.966861
2013  FirstPumpArriving_DeployedFromStation     103055   9933   9.638543
      NumPumpsAttending                         103055    381   0.369705
      PumpSecondsOnSite_max                     103055  10538  10.225608
      PumpSecondsOnSite_mean                    103055  10538  10.225608
      PumpSecondsOnSite_min                     103055  10538  10.225608
      TravelTimeSeconds_max                     103055  11113  10.783562
      TravelTimeSeconds_mean                    103055  11113  10.783562
      TravelTimeSeconds_min                     103055  11113  10.783562
      TurnoutTimeSeconds_max                    103055  11125  10.795206
      TurnoutTimeSeconds_mean 

FirstPumpArriving_DeployedFromStation    121597
PumpSecondsOnSite_min                    151241
PumpSecondsOnSite_mean                   151241
PumpSecondsOnSite_max                    151241
TurnoutTimeSeconds_min                   139247
TurnoutTimeSeconds_mean                  139247
TurnoutTimeSeconds_max                   139247
TravelTimeSeconds_min                    139244
TravelTimeSeconds_mean                   139244
TravelTimeSeconds_max                    139244
dtype: int64 


total ligne na 165791
Total de lignes 1757310
Poucentage 9.434362747608562 %

ligne par année


Nb lignes     Na        %Na
Année Colonne                                                           
2009  FirstPumpArriving_DeployedFromStation     133038  11832   8.893700
      PumpSecondsOnSite_max                     133038  31776  23.884905
      PumpSecondsOnSite_mean                    133038  31776  23.884905
      PumpSecondsOnSite_min                     133038  31776  23.884905
      TravelTimeSeconds_max                     133038  16009  12.033404
      TravelTimeSeconds_mean                    133038  16009  12.033404
      TravelTimeSeconds_min                     133038  16009  12.033404
      TurnoutTimeSeconds_max                    133038  16008  12.032652
      TurnoutTimeSeconds_mean                   133038  16008  12.032652
      TurnoutTimeSeconds_min                    133038  16008  12.032652
2010  FirstPumpArriving_DeployedFromStation     124388  14032  11.280831
      PumpSecondsOnSite_max                     124388  17665  14.201531
      PumpSecondsOnSite_mean                    124388  17665  14.201531
      PumpSecondsOnSite_min                     124388  17665  14.201531
      TravelTimeSeconds_max                     124388  15869  12.757662
      TravelTimeSeconds_mean                    124388  15869  12.757662
      TravelTimeSeconds_min                     124388  15869  12.757662
      TurnoutTimeSeconds_max                    124388  15884  12.769721
      TurnoutTimeSeconds_mean                   124388  15884  12.769721
      TurnoutTimeSeconds_min                    124388  15884  12.769721
2011  FirstPumpArriving_DeployedFromStation     114685  12001  10.464315
      PumpSecondsOnSite_max                     114685  15166  13.224048
      PumpSecondsOnSite_mean                    114685  15166  13.224048
      PumpSecondsOnSite_min                     114685  15166  13.224048
      TravelTimeSeconds_max                     114685  13567  11.829795
      TravelTimeSeconds_mean                    114685  13567  11.829795
      TravelTimeSeconds_min                     114685  13567  11.829795
      TurnoutTimeSeconds_max                    114685  13565  11.828051
      TurnoutTimeSeconds_mean                   114685  13565  11.828051
      TurnoutTimeSeconds_min                    114685  13565  11.828051
2012  FirstPumpArriving_DeployedFromStation     107645  10866  10.094291
      PumpSecondsOnSite_max                     107645  11602  10.778020
      PumpSecondsOnSite_mean                    107645  11602  10.778020
      PumpSecondsOnSite_min                     107645  11602  10.778020
      TravelTimeSeconds_max                     107645  12231  11.362348
      TravelTimeSeconds_mean                    107645  12231  11.362348
      TravelTimeSeconds_min                     107645  12231  11.362348
      TurnoutTimeSeconds_max                    107645  12225  11.356775
      TurnoutTimeSeconds_mean                   107645  12225  11.356775
      TurnoutTimeSeconds_min                    107645  12225  11.356775
2013  FirstPumpArriving_DeployedFromStation     102674   9552   9.303232
      PumpSecondsOnSite_max                     102674  10157   9.892475
      PumpSecondsOnSite_mean                    102674  10157   9.892475
      PumpSecondsOnSite_min                     102674  10157   9.892475
      TravelTimeSeconds_max                     102674  10732  10.452500
      TravelTimeSeconds_mean                    102674  10732  10.452500
      TravelTimeSeconds_min                     102674  10732  10.452500
      TurnoutTimeSeconds_max                    102674  10744  10.464188
      TurnoutTimeSeconds_mean                   102674  10744  10.464188
      TurnoutTimeSeconds_min                    102674  10744  10.464188
2014  FirstPumpArriving_DeployedFromStation      95472   7855   8.227543
      PumpSecondsOnSite_max                      95472   8309   8.703075
      PumpSecondsOnSite_mean                     95472   8309   8.703075
      PumpSecondsOnSite_min   

Series([], dtype: int64) 


total ligne na 0
Total de lignes 1591519
Poucentage 0.0 %

ligne par année


,,Nb lignes,Na,%Na
Année,Colonne,,,


In [ ]:
df.head()
# df[df.CalYear > 2015].info()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1591519 entries, 0 to 1771002
Data columns (total 19 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   IncidentNumber                         1591519 non-null  object 
 1   CalYear                                1591519 non-null  int64  
 2   HourOfCall                             1591519 non-null  int64  
 3   PropertyType                           1591519 non-null  object 
 4   Postcode_district                      1591519 non-null  object 
 5   FirstPumpArriving_DeployedFromStation  1591519 non-null  object 
 6   NumPumpsAttending                      1591519 non-null  float64
 7   Month                                  1591519 non-null  int64  
 8   DayOfWeek                              1591519 non-null  int64  
 9   StopCode                               1591519 non-null  object 
 10  PumpSecondsOnSite_min                  1591519 

In [20]:
df.to_csv("../data/PreProcess.csv", sep=";", index=False)

In [2]:
def ToTextCatgories(df):
    df.CalYear = df.CalYear.astype("str")
    df.HourOfCall = df.HourOfCall.astype("str")
    df.NumPumpsAttending = df.NumPumpsAttending.astype("str")
    df.Month = df.Month.astype("str")
    df.DayOfWeek = df.DayOfWeek.astype("str")
    return df

In [21]:
# ------------------------- Préparation des informations de réduction des valeurs catégorielles -----------------------------

# Traitmeent des catégories. Uniquement des catégories nominales
# possibilité Test Anova,

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/PreProcess.csv", sep=";", low_memory=False)

df = ToTextCatgories(df)

display(df.head())
display(df.info())


,IncidentNumber,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2091,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,N7,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,6091,2009,0,PURPOSE BUILT FLATS/MAISONETTES - UP TO 3 STOREYS,NW5,KENTISH TOWN,2.0,1,4,AFA,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591519 entries, 0 to 1591518
Data columns (total 19 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   IncidentNumber                         1591519 non-null  object 
 1   CalYear                                1591519 non-null  object 
 2   HourOfCall                             1591519 non-null  object 
 3   PropertyType                           1591519 non-null  object 
 4   Postcode_district                      1591519 non-null  object 
 5   FirstPumpArriving_DeployedFromStation  1591519 non-null  object 
 6   NumPumpsAttending                      1591519 non-null  object 
 7   Month                                  1591519 non-null  object 
 8   DayOfWeek                              1591519 non-null  object 
 9   StopCode                               1591519 non-null  object 
 10  PumpSecondsOnSite_min                  159

None

In [23]:
# Traitmeent des catégories. Uniquement des catégories nominales
# possibilité Test Anova,


def ShowValues(col):
    print(col, len(df[col].unique()))
    # print(df[col].value_counts())


cols_categ = [
    "StopCode",
    "PropertyType",
    "Postcode_district",
]

cols_cible = [
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
]

cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]

cols_cible_cat = ["FirstPumpArriving_DeployedFromStation", "NumPumpsAttending"]

# nb de valeurs par catégories
for col in cols_categ:
    ShowValues(col)
print()

df_temp = df[df.CalYear.astype("int") > 2016]

# Sélection de caractéristiques récursive
import statsmodels.api
from statsmodels.api import stats
import statsmodels.formula.api as smf

# Anova sur chaque colonne cible avec toutes les colonnes catégorielles
all_result = []
cols_to_check = "+".join(cols_categ)
for cols in cols_cible:
    cible_result = []
    for col in cols:
        print("Test de", col)
        results = smf.ols(f"{col} ~ {cols_to_check}", data=df_temp).fit()
        # table = stats.anova_lm(result)
        # display(table)
        results_df = pd.DataFrame(
            {
                f"{col} coefficients": round(results.params, 3),
                f"{col} pvalues": round(results.pvalues, 3),
            }
        )
        cible_result.append(results_df)
        # display(results_df.head(30))
        # print("Test de", col, "ok")
    all_result.append(cible_result)

for cible_results in all_result:
    for result in cible_results:
        display(result.head(5))

# merge les result dans un dataFrame
compiled_results = []
for cible_results in all_result:
    cible_result = cible_results[0]
    i = 0
    for result in cible_results:
        if i == 0:
            print("continue")
            i += 1
            continue
        print("merge")
        cible_result = pd.merge(
            cible_result, result, left_index=True, right_index=True, how="outer"
        )
    compiled_results.append(cible_result)

# affiche les df mergés par type de cible
# print("Preview")
# for cible_results in compiled_results:
#     display(cible_results.head(3))


# regarde si la pvalue est la même pour toutes les caétgories : ajout de la colonne Discord : True si il y a des delta
# filtre les lignes avec que des valeurs > 0.05 ou que que des valeurs < 0.05
# pour voir si certaines interviennent que dans certains cas
def TreatDiscordRow(columns, row):
    keeps = []
    nb = 0
    for i in range(1, len(columns), 2):
        # print("col", columns[i])
        # print(row)
        keeps.append(row[columns[i]] <= 0.05)
        nb += 1
    val = not (sum(keeps) == 0) | (sum(keeps) == nb)  # sum(keeps)
    # print(val)
    return val


def TreatKeepRow(columns, row):
    keeps = []
    nb = 0
    for i in range(1, len(columns), 2):
        # print("col", columns[i])
        # print(row[columns[i]])
        keeps.append(row[columns[i]] <= 0.05)
        nb += 1
    val = sum(keeps)
    # si toutes les pvalues sont non significatives
    if val == 0:
        return False
    # si toutes les pvalues sont significatives
    elif val == nb:
        return True
    # sinon, conserve si c'est significatif sur la moyenne
    else:
        mean_col = [s for s in columns if "mean pvalues" in s][0]
        val = row[mean_col]
        # print("mean_col", mean_col)
        # print("mean_col", val)
        return True if val <= 0.05 else False
    # print(val)
    return val


print("Calcule Discords")
for cible_results in compiled_results:
    # display(cible_results.head())
    cible_results["Discord"] = cible_results.apply(
        lambda row: TreatDiscordRow(cible_results.columns, row), axis=1
    )
    cible_results["Keep"] = cible_results.apply(
        lambda row: TreatKeepRow(cible_results.columns, row), axis=1
    )
    display(cible_results[cible_results.Discord == True].head())

for i in range(0, len(compiled_results)):
    print(
        f"{cols_cible_type[i]} {compiled_results[i]["Discord"].sum()} discords, {compiled_results[i]["Keep"].sum()} keeps, sur {len(compiled_results[i])}, "
        f"conserve {round(compiled_results[i]["Keep"].sum() / len(compiled_results[i]) * 100, 0)}%"
    )

StopCode 28
PropertyType 292
Postcode_district 324

Test de PumpSecondsOnSite_min
Test de PumpSecondsOnSite_mean
Test de PumpSecondsOnSite_max
Test de TurnoutTimeSeconds_min
Test de TurnoutTimeSeconds_mean
Test de TurnoutTimeSeconds_max
Test de TravelTimeSeconds_min
Test de TravelTimeSeconds_mean
Test de TravelTimeSeconds_max


,PumpSecondsOnSite_min coefficients,PumpSecondsOnSite_min pvalues
Intercept,720.310,0.245
StopCode[T.CHIMNEY FIRE],2909.027,0.000
StopCode[T.FALSE ALARM - GOOD INTENT],168.508,0.000
StopCode[T.FALSE ALARM - MALICIOUS],107.859,0.000
StopCode[T.LATE CALL],1278.401,0.000


,PumpSecondsOnSite_mean coefficients,PumpSecondsOnSite_mean pvalues
Intercept,539.713,0.405
StopCode[T.CHIMNEY FIRE],3075.949,0.000
StopCode[T.FALSE ALARM - GOOD INTENT],209.920,0.000
StopCode[T.FALSE ALARM - MALICIOUS],122.294,0.000
StopCode[T.LATE CALL],1189.866,0.000


,PumpSecondsOnSite_max coefficients,PumpSecondsOnSite_max pvalues
Intercept,212.292,0.771
StopCode[T.CHIMNEY FIRE],3220.292,0.000
StopCode[T.FALSE ALARM - GOOD INTENT],254.143,0.000
StopCode[T.FALSE ALARM - MALICIOUS],136.048,0.000
StopCode[T.LATE CALL],1086.063,0.000


,TurnoutTimeSeconds_min coefficients,TurnoutTimeSeconds_min pvalues
Intercept,86.776,0.000
StopCode[T.CHIMNEY FIRE],2.490,0.402
StopCode[T.FALSE ALARM - GOOD INTENT],1.652,0.000
StopCode[T.FALSE ALARM - MALICIOUS],3.136,0.000
StopCode[T.LATE CALL],16.542,0.003


...

continue
merge
merge
continue
merge
merge
continue
merge
merge
Calcule Discords


,PumpSecondsOnSite_min coefficients,PumpSecondsOnSite_min pvalues,PumpSecondsOnSite_mean coefficients,PumpSecondsOnSite_mean pvalues,PumpSecondsOnSite_max coefficients,PumpSecondsOnSite_max pvalues,Discord,Keep
Postcode_district[T.KT22],1421.200,0.036,1246.963,0.079,1178.194,0.140,True,False
PropertyType[T.AIRFIELD/RUNWAY],-1064.185,0.007,-606.963,0.142,146.913,0.753,True,False
PropertyType[T.BARBECUE],-948.195,0.007,-706.137,0.055,-430.398,0.298,True,False
PropertyType[T.BARGE],-170.091,0.635,418.967,0.263,1012.678,0.016,True,False
PropertyType[T.BEACH],-816.967,0.049,-445.139,0.305,-46.733,0.924,True,False


,TurnoutTimeSeconds_min coefficients,TurnoutTimeSeconds_min pvalues,TurnoutTimeSeconds_mean coefficients,TurnoutTimeSeconds_mean pvalues,TurnoutTimeSeconds_max coefficients,TurnoutTimeSeconds_max pvalues,Discord,Keep
PropertyType[T.AIRPORT - HANGAR],-9.466,0.372,-26.232,0.015,-42.420,0.001,True,True
PropertyType[T.AIRPORT BUILDING (NOT TERMINAL OR HANGAR)],-18.430,0.067,-30.268,0.003,-40.649,0.001,True,True
PropertyType[T.ANIMAL BOARDING/BREEDING ESTABLISHMENT - CATS],-17.759,0.250,-29.904,0.057,-41.776,0.026,True,False
PropertyType[T.ANIMAL PRODUCTS PROCESSING PLANT],-13.426,0.211,-31.173,0.004,-48.819,0.000,True,True
PropertyType[T.ART GALLERY],-16.059,0.108,-31.125,0.002,-45.886,0.000,True,True


,TravelTimeSeconds_min coefficients,TravelTimeSeconds_min pvalues,TravelTimeSeconds_mean coefficients,TravelTimeSeconds_mean pvalues,TravelTimeSeconds_max coefficients,TravelTimeSeconds_max pvalues,Discord,Keep
Postcode_district[T.BR2],-85.331,0.084,-107.061,0.035,-122.666,0.039,True,True
Postcode_district[T.BR5],-86.223,0.081,-112.040,0.028,-132.425,0.026,True,True
Postcode_district[T.BR6],-95.624,0.053,-115.992,0.023,-130.986,0.028,True,True
Postcode_district[T.CM13],151.751,0.015,140.744,0.029,134.428,0.074,True,True
Postcode_district[T.CM14],115.225,0.027,85.604,0.110,60.684,0.332,True,False


PumpSecondsOnSite 90 discords, 71 keeps, sur 618, conserve 11.0%
TurnoutTimeSeconds 142 discords, 277 keeps, sur 618, conserve 45.0%
TravelTimeSeconds 149 discords, 413 keeps, sur 618, conserve 67.0%


In [40]:
def SplitCatValue(s, return_name):
    result = s.split("[")
    return result[0] if return_name else result[1][2:-1]


# sauve les infos des valeurs catégorielles et leur conservation
for i in range(0, len(compiled_results)):
    df_to_save = compiled_results[i][["Keep"]]
    # supprime la ligne Intercept
    df_to_save = df_to_save.drop(
        df_to_save[df_to_save.index.str.upper() == "INTERCEPT"].index
    )
    # les valeurs de df_keep ont le format
    #           AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION]	True (colonne Keep)
    # crée 2 colonnes, nom de colonne et valeur
    df_to_save["Column"] = df_to_save.index
    df_to_save["Column"] = df_to_save["Column"].apply(lambda s: SplitCatValue(s, True))
    df_to_save["Value"] = df_to_save.index
    df_to_save["Value"] = df_to_save["Value"].apply(lambda s: SplitCatValue(s, False))
    name = f"keep {cols_cible_type[i]}"
    display(df_to_save.head())

    print(name)
    df_to_save.to_csv(f"../data/{name}.csv", sep=";", index=False)

,Keep,Column,Value
Postcode_district[T.BR1],False,Postcode_district,BR1
Postcode_district[T.BR2],False,Postcode_district,BR2
Postcode_district[T.BR3],False,Postcode_district,BR3
Postcode_district[T.BR4],False,Postcode_district,BR4
Postcode_district[T.BR5],False,Postcode_district,BR5


keep PumpSecondsOnSite


,Keep,Column,Value
Postcode_district[T.BR1],False,Postcode_district,BR1
Postcode_district[T.BR2],False,Postcode_district,BR2
Postcode_district[T.BR3],False,Postcode_district,BR3
Postcode_district[T.BR4],False,Postcode_district,BR4
Postcode_district[T.BR5],False,Postcode_district,BR5


keep TurnoutTimeSeconds


,Keep,Column,Value
Postcode_district[T.BR1],True,Postcode_district,BR1
Postcode_district[T.BR2],True,Postcode_district,BR2
Postcode_district[T.BR3],True,Postcode_district,BR3
Postcode_district[T.BR4],False,Postcode_district,BR4
Postcode_district[T.BR5],True,Postcode_district,BR5


keep TravelTimeSeconds


In [26]:
# Vérification de la pertinence des colonnes CalYear HourOfCall Month DayOfWeek sur la valeur moyenne des 3 cibles

df_temp = df[df.CalYear.astype("int") > 2016]

# Anova sur chaque colonne cible avec toutes les colonnes catégorielles
# all_result = []
# cols_to_check = "+".join(cols_categ)
for cols in cols_cible:
    # cible_result = []
    for col in cols:
        if not ("_mean" in col):
            continue
        print("Test de", col)
        results = smf.ols(
            f"{col} ~ CalYear + HourOfCall + Month + DayOfWeek", data=df_temp
        ).fit()
        # display(pd.DataFrame(
        #     {
        #         f"{col} coefficients": round(results.params, 3),
        #         f"{col} pvalues": round(results.pvalues, 3),
        #     }
        # ))
        print(results.summary())
        print()

print("On conserve CalYear + HourOfCall + Month + DayOfWeek")

Test de PumpSecondsOnSite_mean
                              OLS Regression Results                              
Dep. Variable:     PumpSecondsOnSite_mean   R-squared:                       0.003
Model:                                OLS   Adj. R-squared:                  0.003
Method:                     Least Squares   F-statistic:                     49.73
Date:                    Sat, 28 Dec 2024   Prob (F-statistic):               0.00
Time:                            11:01:18   Log-Likelihood:            -7.2550e+06
No. Observations:                  830848   AIC:                         1.451e+07
Df Residuals:                      830800   BIC:                         1.451e+07
Df Model:                              47                                         
Covariance Type:                nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [18]:
# ------------------------- Creation des 3 datasets par cible -----------------------------

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/PreProcess.csv", sep=";", low_memory=False)

# supprime IncidentNumber qui n'est plus nécessaire et ne doit pas être une donnée d'entrainement
df = df.drop(["IncidentNumber"], axis=1)

df = ToTextCatgories(df)

display(df.head(3))
display(df.info())

,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,CAR,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591519 entries, 0 to 1591518
Data columns (total 18 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   CalYear                                1591519 non-null  object 
 1   HourOfCall                             1591519 non-null  object 
 2   PropertyType                           1591519 non-null  object 
 3   Postcode_district                      1591519 non-null  object 
 4   FirstPumpArriving_DeployedFromStation  1591519 non-null  object 
 5   NumPumpsAttending                      1591519 non-null  object 
 6   Month                                  1591519 non-null  object 
 7   DayOfWeek                              1591519 non-null  object 
 8   StopCode                               1591519 non-null  object 
 9   PumpSecondsOnSite_min                  1591519 non-null  float64
 10  PumpSecondsOnSite_mean                 159

None

In [ ]:
cols_categ = [
    "StopCode",
    "PropertyType",
    "Postcode_district",
]


cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]



def ReduceCategValues(df_new, df_keep, col_cible_type):

    display(df_keep.head())
    replaces = {}

    for keep_info in df_keep.itertuples():

        # Ajoute dans le dictionnaire de Map

        if not (keep_info.Keep):
            # replaces.append(keep_info.Value)
            if not (keep_info.Column in replaces):
                print(keep_info.Column)
                replaces[keep_info.Column] = []
            val = replaces[keep_info.Column]
            val.append(keep_info.Value)
            # print("val", val)
            replaces[keep_info.Column] = val
    print("replaces", replaces)
    for item in replaces:
        if (col_cible_type == "TravelTimeSeconds") & (item == "Postcode_district"):
            print("Ignore", col_cible_type, item)
            continue
        print(item)
        print("Avant remplacement", len(df_new[item].unique()), df_new[item].unique())
        print("A remplacer", len(replaces[item]), replaces[item])
        df_new[item] = df_new[item].replace(to_replace=replaces[item], value="OTHER")
        print("Après remplacement", len(df_new[item].unique()), df_new[item].unique())
        print()



# Creation des 3 datasets par cible

for name in cols_cible_type:
    # charge le fichier de keep
    filename = f"keep {name}"
    df_keep = pd.read_csv(f"../data/{filename}.csv", sep=";")
    print(name)
    # crée une copie du dataset
    df_prepare = df.copy()
    # Réduit le nb de valeurs de catégorie pour les 3 colonnes de cols_categ, à partir des infos des fichiers Keep par cible
    ReduceCategValues(df_prepare, df_keep, name)
    display(df_prepare.head(10))
    df_prepare.to_csv(f"../data/df {name}.csv", sep=";", index=False)

PumpSecondsOnSite


,Keep,Column,Value
0,False,Postcode_district,BR1
1,False,Postcode_district,BR2
2,False,Postcode_district,BR3
3,False,Postcode_district,BR4
4,False,Postcode_district,BR5


Postcode_district
PropertyType
replaces {'Postcode_district': ['BR1', 'BR2', 'BR3', 'BR4', 'BR5', 'BR6', 'BR7', 'CM13', 'CM14', 'CR0', 'CR2', 'CR3', 'CR4', 'CR5', 'CR7', 'CR8', 'CR9', 'DA14', 'DA15', 'DA16', 'DA17', 'DA18', 'DA1', 'DA5', 'DA6', 'DA7', 'DA8', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E1W', 'E1', 'E20', 'E22', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'EC1A', 'EC1M', 'EC1N', 'EC1R', 'EC1V', 'EC1Y', 'EC2A', 'EC2M', 'EC2N', 'EC2P', 'EC2R', 'EC2V', 'EC2Y', 'EC3A', 'EC3M', 'EC3N', 'EC3R', 'EC3V', 'EC4A', 'EC4M', 'EC4N', 'EC4R', 'EC4V', 'EC4Y', 'EN1', 'EN2', 'EN3', 'EN4', 'EN5', 'EN6', 'EN7', 'EN8', 'HA0', 'HA1', 'HA2', 'HA3', 'HA4', 'HA5', 'HA6', 'HA7', 'HA8', 'HA9', 'IG11', 'IG1', 'IG2', 'IG3', 'IG4', 'IG5', 'IG6', 'IG7', 'IG8', 'IG9', 'KT17', 'KT18', 'KT19', 'KT1', 'KT22', 'KT2', 'KT3', 'KT4', 'KT5', 'KT6', 'KT8', 'KT9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N1C', 'N1', 'N20', 'N21', 'N22', 'N2', 'N3', 'N4', 'N5'

,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,CAR,OTHER,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,OTHER,OTHER,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,OTHER,OTHER,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,2009,0,OTHER,OTHER,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,2009,0,OTHER,OTHER,KENTISH TOWN,2.0,1,4,AFA,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,2009,0,CAR,OTHER,EAST GREENWICH,1.0,1,4,SST-RTC,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,2009,0,OTHER,OTHER,HESTON,2.0,1,4,SECONDARY FIRE,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,2009,0,SMALL REFUSE/RUBBISH CONTAINER,OTHER,CLERKENWELL,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,2009,0,OTHER,OTHER,EUSTON,1.0,1,4,AFA,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,2009,0,OTHER,OTHER,DOWGATE,1.0,1,4,SST-LIFT RELEASE,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0


TurnoutTimeSeconds


...

,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,CAR,OTHER,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,ROAD SURFACE/PAVEMENT,OTHER,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),OTHER,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,OTHER,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,2009,0,PURPOSE BUILT FLATS/MAISONETTES - UP TO 3 STOREYS,OTHER,KENTISH TOWN,2.0,1,4,AFA,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,2009,0,CAR,OTHER,EAST GREENWICH,1.0,1,4,SST-RTC,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,2009,0,REFUSE/RUBBISH TIP,OTHER,HESTON,2.0,1,4,SECONDARY FIRE,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,2009,0,SMALL REFUSE/RUBBISH CONTAINER,OTHER,CLERKENWELL,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,2009,0,STUDENT HALL OF RESIDENCE,OTHER,EUSTON,1.0,1,4,AFA,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,2009,0,PURPOSE BUILT FLATS/MAISONETTES - 4 TO 9 STOREYS,OTHER,DOWGATE,1.0,1,4,SST-LIFT RELEASE,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0


TravelTimeSeconds


,Keep,Column,Value
0,True,Postcode_district,BR1
1,True,Postcode_district,BR2
2,True,Postcode_district,BR3
3,False,Postcode_district,BR4
4,True,Postcode_district,BR5


Postcode_district
PropertyType
StopCode
replaces {'Postcode_district': ['BR4', 'BR7', 'BR8', 'CM14', 'CR3', 'CR5', 'CR6', 'DA16', 'DA18', 'DA1', 'DA5', 'E22', 'EC2P', 'EN2', 'EN8', 'EN9', 'HA1', 'HA3', 'HA6', 'IG4', 'IG5', 'KT19', 'KT8', 'KT9', 'N11', 'N20', 'OX9', 'RM13', 'RM14', 'RM15', 'RM19', 'RM4', 'SE1P', 'SE22', 'SE28', 'SL0', 'SL3', 'SM7', 'SW13', 'TW15', 'TW16', 'TW19', 'TW5', 'TW6', 'UB6', 'UB7', 'UB9', 'W1K', 'WD23', 'WD6'], 'PropertyType': ['AGRICULTURAL VEHICLE', 'AIRFIELD/RUNWAY', 'AIRPORT - FUEL STORAGE', 'AIRPORT - TERMINAL', 'AIRPORT BUILDING (NOT TERMINAL OR HANGAR)', 'ANIMAL BOARDING/BREEDING ESTABLISHMENT - CATS', 'ANIMAL BOARDING/BREEDING ESTABLISHMENT - DOGS', 'ANIMAL HARM OUTDOORS', 'ATHLETICS STADIUM', 'BARGE', 'BARN', 'BEACH', 'BOARDING SCHOOL ACCOMMODATION', 'BOAT - TANKER', 'BRIDGE', 'BULK GAS STORAGE', 'BULK HAZARDOUS MATERIALS STORAGE', 'BULK OIL STORAGE', 'BULK WASTE STORAGE', 'BUNGALOW - SINGLE OCCUPANCY', 'CABLES', 'CAMPING TENT', 'CANAL/RIVERBANK VEGETA

,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,OTHER,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,OTHER,UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,N7,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,2009,0,OTHER,NW5,KENTISH TOWN,2.0,1,4,AFA,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,2009,0,OTHER,SE3,EAST GREENWICH,1.0,1,4,SST-RTC,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,2009,0,OTHER,TW3,HESTON,2.0,1,4,SECONDARY FIRE,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,2009,0,SMALL REFUSE/RUBBISH CONTAINER,EC1V,CLERKENWELL,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,2009,0,STUDENT HALL OF RESIDENCE,WC1B,EUSTON,1.0,1,4,AFA,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,2009,0,OTHER,EC1A,DOWGATE,1.0,1,4,SST-LIFT RELEASE,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0


In [33]:
# ------------------------- Encodage des 3 datasets par cible -----------------------------

import pandas as pd
import numpy as np

import category_encoders as ce
import joblib  # Pour sauvegarder l'encodeur

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# cols_categ = ["CalYear", "HourOfCall", "Month", "DayOfWeek", "StopCode", "PropertyType", "Postcode_district"]
cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]

for name in cols_cible_type:
    print(name)
    df = pd.read_csv(f"../data/df {name}.csv", sep=";", low_memory=False)
    df = ToTextCatgories(df)
    display(df.head(3))
    # Encodage des catégories

    # CalYear, encodage OrdinalEncoder
    encoder = ce.OrdinalEncoder(cols=["CalYear"] )
    print("OrdinalEncoder")
    df = encoder.fit_transform(df)
    joblib.dump(encoder, f"../data/_ce_{name}_OrdinalEncoder.pkl")

    # Les autres colonnes, encodage binaire
    encoder = ce.BinaryEncoder(cols=["HourOfCall", "Month", "DayOfWeek", "StopCode", "PropertyType", "Postcode_district"])
    print("BinaryEncoder")
    df = encoder.fit_transform(df)
    joblib.dump(encoder, f"../data/_ce_{name}_BinaryEncoder.pkl")

    display(df.head(10))
    df.to_csv(f"../data/_df_ready_{name}.csv", sep=";", index=False)

PumpSecondsOnSite


,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,CAR,OTHER,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,OTHER,OTHER,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,OTHER,OTHER,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0


OrdinalEncoder
BinaryEncoder


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,BATTERSEA,2.0,0,0,0,1,0,0,1,0,0,0,0,1,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,EDMONTON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,HILLINGDON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,HOLLOWAY,2.0,0,0,0,1,0,0,1,0,0,0,1,0,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,KENTISH TOWN,2.0,0,0,0,1,0,0,1,0,0,0,1,1,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,EAST GREENWICH,1.0,0,0,0,1,0,0,1,0,0,0,0,1,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,HESTON,2.0,0,0,0,1,0,0,1,0,0,0,1,0,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,CLERKENWELL,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,EUSTON,1.0,0,0,0,1,0,0,1,0,0,0,1,1,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,DOWGATE,1.0,0,0,0,1,0,0,1,0,0,1,0,0,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0


TurnoutTimeSeconds


...

,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,PropertyType_6,PropertyType_7,PropertyType_8,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,BATTERSEA,2.0,0,0,0,1,0,0,1,0,0,0,0,1,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,EDMONTON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,HILLINGDON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,HOLLOWAY,2.0,0,0,0,1,0,0,1,0,0,0,1,0,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,KENTISH TOWN,2.0,0,0,0,1,0,0,1,0,0,0,1,1,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,EAST GREENWICH,1.0,0,0,0,1,0,0,1,0,0,0,0,1,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,HESTON,2.0,0,0,0,1,0,0,1,0,0,0,1,0,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,CLERKENWELL,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,EUSTON,1.0,0,0,0,1,0,0,1,0,0,0,1,1,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,DOWGATE,1.0,0,0,0,1,0,0,1,0,0,1,0,0,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0


TravelTimeSeconds


,CalYear,HourOfCall,PropertyType,Postcode_district,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,2009,0,OTHER,SW11,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,2009,0,ROAD SURFACE/PAVEMENT,N9,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,2009,0,OTHER,UB10,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0


OrdinalEncoder
BinaryEncoder


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,PropertyType_6,PropertyType_7,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Postcode_district_5,Postcode_district_6,Postcode_district_7,Postcode_district_8,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,BATTERSEA,2.0,0,0,0,1,0,0,1,0,0,0,0,1,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,EDMONTON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,HILLINGDON,1.0,0,0,0,1,0,0,1,0,0,0,1,0,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
3,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,HOLLOWAY,2.0,0,0,0,1,0,0,1,0,0,0,1,0,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,KENTISH TOWN,2.0,0,0,0,1,0,0,1,0,0,0,1,1,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
5,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,EAST GREENWICH,1.0,0,0,0,1,0,0,1,0,0,0,0,1,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0
6,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,HESTON,2.0,0,0,0,1,0,0,1,0,0,0,1,0,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
7,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,CLERKENWELL,1.0,0,0,0,1,0,0,1,0,0,0,1,0,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
8,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,EUSTON,1.0,0,0,0,1,0,0,1,0,0,0,1,1,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0
9,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,DOWGATE,1.0,0,0,0,1,0,0,1,0,0,1,0,0,600.0,600.0,600.0,129.0,129.0,129.0,567.0,567.0,567.0
